The old ramp scored sidewalks with the Red ramp points highlighted

In [2]:
import json
import geojson
from shapely.geometry import shape, Point, Polygon
import geopandas as gpd
import folium


ramps_gdf = gpd.read_file('../data/Ramps New.json')
sidewalks_gdf = gpd.read_file('../data/Sidewalks.json')
ramps_gdf = ramps_gdf.to_crs(sidewalks_gdf.crs)
ramps_gdf.geometry = ramps_gdf.geometry.buffer(5)

In [10]:
ramps_gdf = ramps_gdf[ramps_gdf['score_1'] <=20 ]
point_gdf = ramps_gdf.loc[:, ['FID','score_1','geometry']]
point_gdf.geometry = point_gdf.geometry.buffer(10)
print(point_gdf)



         FID  score_1                                           geometry
34        34     10.0  POLYGON ((768159.863 2952402.649, 768159.863 2...
44        44     10.0  POLYGON ((777964.718 2952597.815, 777964.718 2...
47        47     10.0  POLYGON ((756467.654 2924866.408, 756467.654 2...
49        49     10.0  POLYGON ((756423.282 2924905.711, 756423.282 2...
138      138     10.0  POLYGON ((774896.339 2954017.682, 774896.339 2...
...      ...      ...                                                ...
26207  26207      0.0  POLYGON ((778766.451 2953750.680, 778766.451 2...
26208  26208      0.0  POLYGON ((779347.991 2953680.894, 779347.991 2...
26209  26209      0.0  POLYGON ((779345.727 2953682.906, 779345.727 2...
26210  26210      0.0  POLYGON ((779392.425 2953655.824, 779392.425 2...
26211  26211      0.0  POLYGON ((774055.815 2943855.194, 774055.815 2...

[4239 rows x 3 columns]


In [3]:

joined_gdf = gpd.sjoin(sidewalks_gdf, ramps_gdf, op='intersects')
grouped_gdf = joined_gdf.groupby('SWK_ID').agg({'FID_left': list, 'score_1': 'mean'}).reset_index()
result_gdf = sidewalks_gdf.merge(grouped_gdf, on='SWK_ID', how='left')
print(result_gdf.head())

c:\Users\xiao\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3382: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


   FID SWK_ID MATERIAL SWK_WIDTH          DISTRICT SWK_AREA  PARENT  \
0    0      1       CC       5.5       CHARLESTOWN     3648  ALBIO1   
1    1    402       CC         6  SOUTH DORCHESTER     1671  ALMON1   
2    2    403       CC         6  SOUTH DORCHESTER     1351  ALMON1   
3    3    404       CC         6  SOUTH DORCHESTER     1617  ALMON1   
4    4    405       CC         6  SOUTH DORCHESTER     3795  ALMON1   

        SEG_ID   SIDE  CG_ID  Rpr_Yr  New_SCI  Recon_Date  HPNETWORK  \
0     ALBIO1_0   LEFT  24842       0       85         NaN          0   
1  ALMON1_1749   LEFT    401       0       55         NaN          1   
2  ALMON1_2047  RIGHT    402       0       67         NaN          1   
3  ALMON1_2047   LEFT    403       0       81         NaN          1   
4  ALMON1_2338  RIGHT    404    2015       66         NaN          1   

   GOODCANDID                                           geometry FID_left  \
0           0  POLYGON ((772436.303 2964399.527, 772434.783 2..

In [11]:


new_gdf = result_gdf
# Create a map centered at a specific location
m = folium.Map(location=[42.361145, -71.057083], zoom_start=30)

# Add sidewalks layer to the map with color ramp based on average score
folium.Choropleth(
    geo_data=new_gdf,
    name='Average Score',
    data=new_gdf,
    columns=['SWK_ID', 'score_1'],
    key_on='properties.SWK_ID',
    fill_color='RdYlBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Average Score',
    reverse=True,
    highlight=True,
    show=True
).add_to(m)

#Why there are no geometry data
print(new_gdf['score_1'].isna().sum())
new_gdf = new_gdf.dropna(subset=['geometry'])

# Adds the ability to click the sidewalk to see the score for that sidewalk
folium.GeoJson(
    new_gdf,
    name='Sidewalk Scores',
    tooltip=folium.features.GeoJsonTooltip(fields=['SWK_ID','score_1'], labels=True, sticky=True),
    style_function=lambda feature: {'weight': 1, 'color':'black', 'fillOpacity':0},
    popup=folium.features.GeoJsonPopup(fields=['SWK_ID','score_1'], labels=True, sticky=True),
    show=True
).add_to(m)

folium.GeoJson(
    point_gdf,
    name='Ramp Scores',
    tooltip=folium.features.GeoJsonTooltip(fields=['FID','score_1'], labels=True, sticky=True),
    style_function=lambda feature: {'weight': 3, 'color':'red', 'fillOpacity':1},
    popup=folium.features.GeoJsonPopup(fields=['FID','score_1'], labels=True, sticky=True),
    show=False
).add_to(m)


# Add a layer control to the map
folium.LayerControl().add_to(m)

# Save the map as an HTML file
m.save('maps/sidewalks_map_points.html')

4256
